##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time

### Download the dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/KindlePreprocessed.txt'

### Read the data


In [ ]:
# read and decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text = text.replace('\n', ' ')
print(f'Length of text: {len(text)} characters')

Length of text: 10172244 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

silly predictable romance, read it in an afternoon with a hangover, that's all I'd recommend this author for. Fortunately it was free and I test drove my new Kindle. This is not a hockey romance! It is Completely misleading and not a great story. I a


In [ ]:
# unique characters 
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

96 unique characters


## Process the text

In [ ]:
# convert strings to numerical representation 
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
# invert representation to recover readable strings
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
# join chars into strings
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(10172244,), dtype=int64, numpy=array([85, 75, 78, ..., 84, 16,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

s
i
l
l
y
 
p
r
e
d


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b's' b'i' b'l' b'l' b'y' b' ' b'p' b'r' b'e' b'd' b'i' b'c' b't' b'a'
 b'b' b'l' b'e' b' ' b'r' b'o' b'm' b'a' b'n' b'c' b'e' b',' b' ' b'r'
 b'e' b'a' b'd' b' ' b'i' b't' b' ' b'i' b'n' b' ' b'a' b'n' b' ' b'a'
 b'f' b't' b'e' b'r' b'n' b'o' b'o' b'n' b' ' b'w' b'i' b't' b'h' b' '
 b'a' b' ' b'h' b'a' b'n' b'g' b'o' b'v' b'e' b'r' b',' b' ' b't' b'h'
 b'a' b't' b"'" b's' b' ' b'a' b'l' b'l' b' ' b'I' b"'" b'd' b' ' b'r'
 b'e' b'c' b'o' b'm' b'm' b'e' b'n' b'd' b' ' b't' b'h' b'i' b's' b' '
 b'a' b'u' b't'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"silly predictable romance, read it in an afternoon with a hangover, that's all I'd recommend this aut"
b'hor for. Fortunately it was free and I test drove my new Kindle. This is not a hockey romance! It is '
b'Completely misleading and not a great story. I am Uberly annoyed right now. This book is a disaster. '
b'It is poorly organized; filled with numerous typographical, grammar, and spelling errors; and contain'
b"s some glaring inaccuracies. Let's start with the organization. The first chapter begins with an outl"


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [1]:
# function that takes a sequence, duplicates and shifts it to align the input and label
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [3]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"silly predictable romance, read it in an afternoon with a hangover, that's all I'd recommend this au"
Target: b"illy predictable romance, read it in an afternoon with a hangover, that's all I'd recommend this aut"


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# create training batches w shuffled data to. feed into model
BATCH_SIZE = 64

# buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# length of the vocabulary in chars
vocab_size = len(vocab)
# embedding dimension
embedding_dim = 256
# number of RNN units
rnn_units = 1024

In [ ]:
# build the model (keras.Model subclass)
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  24832     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([76, 19,  3, 68, 60, 75, 72, 44, 58, 49, 51, 63, 41, 76, 82, 11, 73,
       33, 24, 92, 51, 61, 15, 96, 11, 69,  9, 76,  5, 87, 53, 57, 56, 45,
       85, 44,  5, 34, 59, 14, 73, 26, 67, 77, 61, 24, 86, 93, 13, 39, 54,
       40, 16, 48, 73, 15, 63, 34, 20,  8, 88, 19, 91, 28, 52, 85, 33, 70,
       45, 82, 12, 20, 82, 23, 94, 64, 63, 61, 35, 18, 82, 28, 52, 19, 31,
       96, 25, 70, 75, 40, 75, 58, 24,  7, 58, 10, 74, 33, 29, 52])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'e was about to be raped by her boyfriend and possibly his friends and Elijah kills them all and take'

Next Char Predictions:
 b"j1!bZifJXOQ]Gjp)g?6zQ[-~)c'j#uSWVKsJ#@Y,g8ak[6t{+ETF.Ng-]@2&v1y:Rs?dKp*2p5|^][A0p:R1=~7diFiX6%X(h?;R"


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
# attach an optimizer and loss function
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.575968


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
# newly initialized model should have an exponential mean loss approx equal to vocab size
tf.exp(mean_loss).numpy()

97.12198

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# configure checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
# train the model
EPOCHS = 10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
1573/1573 [==============================] - 219s 137ms/step - loss: 1.5956
Epoch 2/10
1573/1573 [==============================] - 217s 137ms/step - loss: 1.2173
Epoch 3/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.1575
Epoch 4/10
1573/1573 [==============================] - 217s 137ms/step - loss: 1.1247
Epoch 5/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.1023
Epoch 6/10
1573/1573 [==============================] - 217s 137ms/step - loss: 1.0864
Epoch 7/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.0748
Epoch 8/10
1573/1573 [==============================] - 217s 137ms/step - loss: 1.0668
Epoch 9/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.0612
Epoch 10/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.0582


## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
# single step prediction
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()


    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
BATCH_SIZE = 5
PREDICT_LEN = 300

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard.h5')

# We seed the model with our initial string, copied BATCH_SIZE times

seed_txt = 'the bottle of wine '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]
  
  # sample from our output distribution
  next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
  print('PREDICTION %d\n\n' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  generated = ''.join([chr(c) for c in p])
  print(generated)
  print()
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

NameError: ignored

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['The book:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), ' ' + '_'*80)
print('\nRun time:', end - start)

The book:  "Ground".  I believe that "Ensemand People" painters as partnarbia. Raw needed Clan of a Til Austing Children. It made me want to read more to mendage struggles and start  by wickers, and I was actually happy.  Incidentabilityation sounds. A great author but nonethelvestful.  I really enjoyed the Uncle had moments of true love, she gets better and I enjoyed the story that were HOT.  This was a sfool in exchange of mystery right with a HEA. I couldn't get enough story! she was a hwitched autobody blog recovery "Awework" and archeofrobber, "years", sweet and shallow. I think in it! Anyway she is finally getting into a man who goes to Beauforms fails into much of loving one compilatular story of a tangled indulging barged picture out of telling you that they didn't know what he wants and be carrier...now he was home, but she helps twice built around art is out to guide. The way he writes with Amish brain freshly nicely the author that move too harsh, which was all healthy backg

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(['The book:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

The book:ave
been
a
hard
enough
craziest
and
sweet
love
interest
.
I
didn't
know
what
Jasping
is
wronging
the


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

1573/1573 [==============================] - 219s 137ms/step - loss: 1.6016


In [ ]:
# generate 10 product reviews using the trained model
start = time.time()
states = None
next_char = tf.constant(['The book', 'The book', 'The book', 'The book', 'The book', 'The book', 'The book', 'The book', 'The book', 'The book'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
words = tf.strings.join(result).numpy().decode("utf-8")
print(result, ' ' + '_'*80)
print('\nRun time:', end - start)
print(words)
text_file = open("/content/drive/MyDrive/Colab Notebooks/results.txt", "w")
n = text_file.write(words)
text_file.close()

tf.Tensor(
[b"The book is subtised. Until I just made Edjortice young pleasure against what the hell. The majority only be so far of Barrow's the same protection eventually seem to want to comment where they are.  Lilliana Timgi wakes compassion to help you hooked revenged on another to torn motthe Russ! And I zouth competerly escape, to the point or the storyline. This is a wonderful story of love that Vic Felton has been with him, he's supposed to be the verge of town of ones jule and sets out to just en if she. Looking forward to reading more books in the series! I will tell about Geri's Justice Hut, forth; this one also thinks so that could take a colded weekend more clever.  She meets the romantic aftermath for a first love and turn down everything finally feeling for Abra.  Would have told her to stop Rexen in for an honesty one.  The two punchs make me a whole lot of twists and turns that are looking for being bride. I really enjoyed this book. I was not disappointed in the next

In [ ]:
arr=words.split("The book")
print(len(arr))

12


Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 1.2598
Epoch 1 Batch 50 Loss 1.1951
Epoch 1 Batch 100 Loss 1.2656
Epoch 1 Batch 150 Loss 1.1717
Epoch 1 Batch 200 Loss 1.2082
Epoch 1 Batch 250 Loss 1.1799
Epoch 1 Batch 300 Loss 1.1402
Epoch 1 Batch 350 Loss 1.1670
Epoch 1 Batch 400 Loss 1.1598
Epoch 1 Batch 450 Loss 1.1074
Epoch 1 Batch 500 Loss 1.1641
Epoch 1 Batch 550 Loss 1.1866
Epoch 1 Batch 600 Loss 1.1654
Epoch 1 Batch 650 Loss 1.1003
Epoch 1 Batch 700 Loss 1.1951
Epoch 1 Batch 750 Loss 1.1966
Epoch 1 Batch 800 Loss 1.1693
Epoch 1 Batch 850 Loss 1.2061
Epoch 1 Batch 900 Loss 1.1839
Epoch 1 Batch 950 Loss 1.1348
Epoch 1 Batch 1000 Loss 1.1722
Epoch 1 Batch 1050 Loss 1.2180
Epoch 1 Batch 1100 Loss 1.2229
Epoch 1 Batch 1150 Loss 1.1078
Epoch 1 Batch 1200 Loss 1.1515
Epoch 1 Batch 1250 Loss 1.1711
Epoch 1 Batch 1300 Loss 1.1207
Epoch 1 Batch 1350 Loss 1.1388
Epoch 1 Batch 1400 Loss 1.1549
Epoch 1 Batch 1450 Loss 1.1069
Epoch 1 Batch 1500 Loss 1.1665
Epoch 1 Batch 1550 Loss 1.1537
Epoch 1 Batch 1600 Loss 1.1267
